# Mô tả các bước chính của thuật toán Actor-Critic với DPG
# 1. **Xây dựng mô hình Actor và Critic**:
#    - Actor: Dự đoán hành động tối ưu dựa trên trạng thái hiện tại.
#    - Critic: Đánh giá giá trị của hành động đó thông qua Q-value.
#    - Target Networks: Các bản sao của Actor và Critic được cập nhật chậm để ổn định quá trình huấn luyện.
# 2. **Replay Buffer**:
#    - Lưu trữ lịch sử các trạng thái, hành động, phần thưởng để huấn luyện mô hình theo batch.
# 3. **Chọn hành động và thám hiểm**:
#    - Actor dự đoán hành động.
#    - Thêm nhiễu Gaussian để khuyến khích thám hiểm các hành động khác nhau.
# 4. **Huấn luyện Critic**:
#    - Sử dụng Replay Buffer để lấy dữ liệu.
#    - Tính toán Q-value mục tiêu từ mạng Target Critic.
#    - Cập nhật mạng Critic để dự đoán Q-value chính xác hơn.
# 5. **Huấn luyện Actor**:
#    - Dựa vào Critic để tìm hướng dẫn Actor chọn hành động tối ưu.
#    - Cập nhật mạng Actor.
# 6. **Cập nhật Target Networks**:
#    - Mạng Target Actor và Target Critic được cập nhật dần dần để theo kịp các thay đổi của Actor và Critic chính.
# 7. **Theo dõi hiệu suất**:
#    - Theo dõi phần thưởng trung bình động để đánh giá tiến trình.
#    - Giảm độ nhiễu để tăng khả năng khai thác (exploitation) khi mô hình đã ổn định.
# 8. **Dừng huấn luyện**:
#    - Dừng lại khi phần thưởng trung bình đạt đến ngưỡng mục tiêu hoặc khi đạt đến số lượng vòng lặp tối đa.

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import gym
import numpy as np
import keras
from keras import ops
from keras import layers
import tensorflow as tf

In [ ]:
# ====================== Thiết lập môi trường ===========================
env = gym.make("CartPole-v1")
num_inputs = 4 # Số lượng đầu vào (trạng thái của môi trường
num_actions = 1 # Số lượng đầu ra (hành động được thực hiện)
num_hidden = 256  # Số lượng nút trong lớp ẩn của mô hình.
gamma = 0.99 # Hệ số chiết khấu để tính giá trị Q.
exploration_noise_std = 0.3  # Khởi đầu với nhiễu cao
min_exploration_noise_std = 0.05  # Giảm dần đến giá trị tối thiểu
noise_decay = 0.995  # Tốc độ giảm nhiễu
max_steps_per_episode = 500
max_episodes = 1000

In [ ]:
# ====================== Xây dựng Actor và Critic =======================
# **Bước 1: Khởi tạo Actor**
# Actor dự đoán hành động dựa trên trạng thái đầu vào
actor_inputs = layers.Input(shape=(num_inputs,))
actor_hidden = layers.Dense(num_hidden, activation="relu")(actor_inputs)
actor_hidden = layers.Dense(num_hidden, activation="relu")(actor_hidden)
actor_output = layers.Dense(num_actions, activation="tanh")(actor_hidden)
actor_model = tf.keras.Model(inputs=actor_inputs, outputs=actor_output)

# **Bước 2: Khởi tạo Critic**
# Critic đánh giá giá trị Q (Q-value) từ trạng thái và hành động
critic_inputs = layers.Input(shape=(num_inputs + num_actions,))
critic_hidden = layers.Dense(num_hidden, activation="relu")(critic_inputs)
critic_hidden = layers.Dense(num_hidden, activation="relu")(critic_hidden)
q_value = layers.Dense(1)(critic_hidden)
critic_model = tf.keras.Model(inputs=critic_inputs, outputs=q_value)

# **Bước 3: Tạo Target Networks**
# Target Networks sử dụng để ổn định quá trình huấn luyện bằng cách chậm cập nhật so với mạng chính
target_actor_model = tf.keras.models.clone_model(actor_model)
target_critic_model = tf.keras.models.clone_model(critic_model)

# ====================== Trình tối ưu hóa =======================
# **Bước 4: Định nghĩa bộ tối ưu hóa**
# Sử dụng Adam optimizer cho Actor và Critic
actor_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
# ====================== Replay Buffer =======================
# **Bước 5: Khởi tạo Replay Buffer**
# Replay Buffer lưu trữ lịch sử trạng thái, hành động, phần thưởng để huấn luyện theo batch
class ReplayBuffer:
    def __init__(self, buffer_capacity=200000, batch_size=128):
        self.buffer_capacity = buffer_capacity
        self.batch_size = batch_size
        self.buffer_counter = 0
        self.state_buffer = np.zeros((buffer_capacity, num_inputs))
        self.action_buffer = np.zeros((buffer_capacity, num_actions))
        self.reward_buffer = np.zeros((buffer_capacity, 1))
        self.next_state_buffer = np.zeros((buffer_capacity, num_inputs))
        self.done_buffer = np.zeros((buffer_capacity, 1))

    # Lưu trữ trải nghiệm (state, action, reward, next_state, done)
    def store(self, state, action, reward, next_state, done):
        index = self.buffer_counter % self.buffer_capacity
        self.state_buffer[index] = state
        self.action_buffer[index] = action
        self.reward_buffer[index] = reward
        self.next_state_buffer[index] = next_state
        self.done_buffer[index] = done
        self.buffer_counter += 1

    # Lấy mẫu batch ngẫu nhiên từ Replay Buffer
    def sample(self):
        max_buffer = min(self.buffer_counter, self.buffer_capacity)
        batch_indices = np.random.choice(max_buffer, self.batch_size)
        return (
            self.state_buffer[batch_indices],
            self.action_buffer[batch_indices],
            self.reward_buffer[batch_indices],
            self.next_state_buffer[batch_indices],
            self.done_buffer[batch_indices],
        )

buffer = ReplayBuffer()

In [ ]:
# ====================== Vòng lặp huấn luyện ===========================
# **Bước 6: Vòng lặp huấn luyện Actor-Critic**
running_reward = 0
tau = 0.005  # Hệ số cập nhật target network

for episode in range(max_episodes):
    # Đặt lại môi trường và trạng thái ban đầu
    state = env.reset()
    episode_reward = 0

    for step in range(max_steps_per_episode):
        # Chuyển trạng thái sang tensor
        state_tensor = tf.convert_to_tensor(state, dtype=tf.float32)
        state_tensor = tf.expand_dims(state_tensor, axis=0)

        # **Actor chọn hành động**
        # Thêm nhiễu Gaussian để khuyến khích thám hiểm
        action = actor_model(state_tensor).numpy()[0]
        action += np.random.normal(0, exploration_noise_std)
        action = np.clip(action, -1, 1)

        # **Thực hiện hành động và lưu trữ trải nghiệm**
        next_state, reward, done, _ = env.step(int(action > 0))
        buffer.store(state, action, reward, next_state, done)
        episode_reward += reward

        # **Huấn luyện Actor và Critic khi Replay Buffer đủ dữ liệu**
        if buffer.buffer_counter >= buffer.batch_size:
            states, actions, rewards, next_states, dones = buffer.sample()
            states = tf.convert_to_tensor(states, dtype=tf.float32)
            actions = tf.convert_to_tensor(actions, dtype=tf.float32)
            rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
            next_states = tf.convert_to_tensor(next_states, dtype=tf.float32)
            dones = tf.convert_to_tensor(dones, dtype=tf.float32)

            # **Huấn luyện Critic**
            with tf.GradientTape() as tape:
                # Tính Q-value mục tiêu từ Target Networks
                target_actions = target_actor_model(next_states)
                target_q_values = rewards + (1 - dones) * gamma * target_critic_model(
                    tf.concat([next_states, target_actions], axis=1)
                )
                # Tính Q-value dự đoán từ Critic model
                predicted_q_values = critic_model(tf.concat([states, actions], axis=1))
                critic_loss = tf.keras.losses.MSE(target_q_values, predicted_q_values)

            # Cập nhật trọng số Critic
            critic_grads = tape.gradient(critic_loss, critic_model.trainable_variables)
            critic_optimizer.apply_gradients(zip(critic_grads, critic_model.trainable_variables))

            # **Huấn luyện Actor**
            with tf.GradientTape() as tape:
                actions_pred = actor_model(states)
                actor_loss = -tf.reduce_mean(critic_model(tf.concat([states, actions_pred], axis=1)))

            # Cập nhật trọng số Actor
            actor_grads = tape.gradient(actor_loss, actor_model.trainable_variables)
            actor_optimizer.apply_gradients(zip(actor_grads, actor_model.trainable_variables))

            # **Cập nhật Target Networks**
            for target_param, param in zip(target_critic_model.trainable_variables, critic_model.trainable_variables):
                target_param.assign(tau * param + (1 - tau) * target_param)

            for target_param, param in zip(target_actor_model.trainable_variables, actor_model.trainable_variables):
                target_param.assign(tau * param + (1 - tau) * target_param)

        # Kiểm tra nếu trạng thái cuối cùng đạt đến điểm dừng
        if done:
            break

        # Cập nhật trạng thái
        state = next_state

    # **Giảm độ nhiễu theo thời gian để khuyến khích khai thác**
    exploration_noise_std = max(min_exploration_noise_std, exploration_noise_std * noise_decay)

    # **Theo dõi phần thưởng trung bình động**
    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

    # In log sau mỗi 10 episode
    if episode % 10 == 0:
        print(f"Episode: {episode}, Running Reward: {running_reward:.2f}, Exploration Noise: {exploration_noise_std:.3f}")

    # **Dừng khi đạt mục tiêu phần thưởng**
    if running_reward > 475:
        print(f"Solved at episode {episode}!")
        break

<ipython-input-14-266f553b5683>:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_state, reward, done, _ = env.step(int(action > 0))


Episode: 0, Running Reward: 0.85, Exploration Noise: 0.298
Episode: 10, Running Reward: 7.09, Exploration Noise: 0.284
Episode: 20, Running Reward: 8.09, Exploration Noise: 0.270
Episode: 30, Running Reward: 8.65, Exploration Noise: 0.257
Episode: 40, Running Reward: 8.96, Exploration Noise: 0.244
Episode: 50, Running Reward: 9.10, Exploration Noise: 0.232
Episode: 60, Running Reward: 9.22, Exploration Noise: 0.221
Episode: 70, Running Reward: 9.32, Exploration Noise: 0.210
Episode: 80, Running Reward: 9.06, Exploration Noise: 0.200
Episode: 90, Running Reward: 9.10, Exploration Noise: 0.190
Episode: 100, Running Reward: 9.06, Exploration Noise: 0.181
Episode: 110, Running Reward: 9.17, Exploration Noise: 0.172
Episode: 120, Running Reward: 9.15, Exploration Noise: 0.164
Episode: 130, Running Reward: 9.17, Exploration Noise: 0.156
Episode: 140, Running Reward: 9.14, Exploration Noise: 0.148
Episode: 150, Running Reward: 9.32, Exploration Noise: 0.141
Episode: 160, Running Reward: 9.36,

Exploration Noise trong code trên được sử dụng để khuyến khích Agent thử nghiệm các hành động đa dạng hơn, giúp khám phá môi trường hiệu quả hơn trong giai đoạn đầu huấn luyện. Nó giúp tránh việc Agent bị mắc kẹt trong các chiến lược không tối ưu do chưa đủ thông tin về môi trường. Khi huấn luyện tiến triển, độ nhiễu giảm dần để chuyển từ thám hiểm (exploration) sang khai thác (exploitation), tập trung vào các hành động tối ưu đã học được. Điều này đảm bảo sự cân bằng giữa việc khám phá và khai thác, cải thiện hiệu quả học tập của mô hình.

